## Importing Libraries

In [1]:
seed_value = 0
import os
os.environ['PYTHONHASHSEED'] = str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)


import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)

from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain 

from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers

## Reading Pre-Processed Data

In [2]:
df = pd.read_csv("Data/Pre_Processed/training.csv")
# df = pd.read_csv("Data/Pre_Processed/validation.csv")
# df = pd.read_csv("Data/Pre_Processed/testing.csv")

In [3]:
df

,Unnamed: 0,Unnamed: 0.1,Date,Open,High,Low,Close,Adj Close,Volume,result,...,wma20,tema20,william20,cci20,cmo20,macd20,macdsignal20,macdhist20,ppo20,parabolicsar20
0,0,0,2010-01-04,545.549988,545.549988,510.924988,537.750000,457.306488,35040012.0,0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,-12345.0,-12345.0,0.0,0.000000
1,1,1,2010-01-05,574.950012,574.950012,532.700012,535.349976,455.265503,9780080.0,0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,-12345.0,-12345.0,0.0,0.999967
2,2,2,2010-01-06,539.950012,547.250000,535.325012,544.000000,462.621460,10831076.0,0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,-12345.0,-12345.0,0.0,0.999967
3,3,3,2010-01-07,544.000000,557.500000,539.000000,553.025024,470.296417,11976808.0,0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,-12345.0,-12345.0,0.0,0.999967
4,4,4,2010-01-08,554.000000,556.900024,548.000000,551.575012,469.063354,6907852.0,0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,-12345.0,-12345.0,0.0,0.999967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727,1727,1727,2016-12-26,527.174988,533.000000,522.325012,523.724976,512.049377,7004598.0,0,...,0.0,0.0,0.305459,0.854843,0.0,0.0,-12345.0,-12345.0,0.0,0.999984
1728,1728,1728,2016-12-27,523.924988,533.549988,523.500000,532.349976,520.482178,7974284.0,2,...,0.0,0.0,0.334807,0.870977,0.0,0.0,-12345.0,-12345.0,0.0,0.999984
1729,1729,1729,2016-12-28,533.950012,534.775024,523.549988,524.849976,513.149353,4405854.0,0,...,0.0,0.0,0.309534,0.848161,0.0,0.0,-12345.0,-12345.0,0.0,0.999984
1730,1730,1730,2016-12-29,525.000000,534.400024,524.750000,532.724976,520.848816,4010412.0,2,...,0.0,0.0,0.335154,0.864246,0.0,0.0,-12345.0,-12345.0,0.0,0.999984


## Creating Array from normalized Technical Indicators

In [4]:
rsi = df[['rsi'+str(n) for n in range(6,21)]].to_numpy()
roc = df[['roc'+str(n) for n in range(6,21)]].to_numpy()
sma= df[['sma'+str(n) for n in range(6,21)]].to_numpy()
ema = df[['ema'+str(n) for n in range(6,21)]].to_numpy()
wma = df[['wma'+str(n) for n in range(6,21)]].to_numpy()
tema = df[['tema'+str(n) for n in range(6,21)]].to_numpy()
william = df[['william'+str(n) for n in range(6,21)]].to_numpy()
cci = df[['cci'+str(n) for n in range(6,21)]].to_numpy()
cmo = df[['cmo'+str(n) for n in range(6,21)]].to_numpy()
macd = df[['macd'+str(n) for n in range(6,21)]].to_numpy()
ppo = df[['ppo'+str(n) for n in range(6,21)]].to_numpy()
parabolicsar = df[['parabolicsar'+str(n) for n in range(6,21)]].to_numpy()

In [5]:
train_images = [[j for j in chain.from_iterable([rsi[i],roc[i],sma[i],ema[i],william[i],cci[i],cmo[i],macd[i],ppo[i],tema[i],wma[i],parabolicsar[i]])]for i in df.index]

In [6]:
train_images = np.array(train_images)
train_images = train_images.reshape(df.shape[0],12,15)

## Creating and Saving Images 

In [7]:
from PIL import Image
l = 0
m = 0
n = 0
df_labels = df['result']

for i in range (0,df.shape[0]):
    
    if(df_labels[i] == 0):    # for hold
        x = train_images[i,:,:]
        x.reshape(12,15)
        array = np.array(x*255).astype(np.uint8)
        image = Image.fromarray(array)
        l = l+1
        img_name = 'hold'+str(l)
        print(img_name)
        image.save('Images\Training\Hold\{}.png'.format(img_name))
#         image.save('Images\Testing\Hold\{}.png'.format(img_name))
#         image.save('Images\Validation\Hold\{}.png'.format(img_name))
        display(image)
        
    if(df_labels[i] == 1):    # for buy
        x = train_images[i,:,:]
        x.reshape(12,15)
        array = np.array(x*255).astype(np.uint8)
        image = Image.fromarray(array)
        m = m+1
        img_name = 'buy'+str(m)
        print(img_name)
        image.save('Images\Training\Buy\{}.png'.format(img_name))
#         image.save('Images\Testing\Buy\{}.png'.format(img_name))
#         image.save('Images\Validation\Buy\{}.png'.format(img_name))
        display(image)
        
    if(df_labels[i] == 2):    # for sell
        x = train_images[i,:,:]
        x.reshape(12,15)
        array = np.array(x*255).astype(np.uint8)
        image = Image.fromarray(array)
        n = n+1
        img_name = 'sell'+str(n)
        print(img_name)
        image.save('Images\Training\Sell\{}.png'.format(img_name))
#         image.save('Images\Testing\Sell\{}.png'.format(img_name))
#         image.save('Images\Validation\Sell\{}.png'.format(img_name))
        display(image)

print(f"total hold image is: {l}")
print(f"total buy image is: {m}")
print(f"total sell image is: {n}")

hold1


hold2


hold3


hold4


hold5


...

sell465


hold826


sell466


hold827


total hold image is: 827
total buy image is: 439
total sell image is: 466
